In [ ]:
import os, sys
import pandas as pd
from sklearn.manifold import MDS
from sklearn.cluster import KMeans

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)


from  techminer import *

# import_scopus_file()
import_references()

In [ ]:
import pandas as pd

documents = pd.read_csv("documents.csv")
documents = documents.sort_values("local_citations", ascending=False)
for row in documents.head(20).iterrows():
    print(row[1]["local_citations"], row[1]["document_title"])
    

In [ ]:
import pandas as pd

references = pd.read_csv("references.csv")
references = references.sort_values("local_citations", ascending=False)
for row in references.head(20).iterrows():
    print(row[1]["local_citations"], row[1]["document_title"])

In [ ]:
pd.read_csv("references.csv").local_citations.sum()